# Intro to data science project

## Data 
Truck GPS 1 day
## Objective
1) หา Origin Destination ของรถบรรทุกในประเทศไทย
2) หา ปริมาณคาร์บอนที่ปล่อยจากการขนส่ง กรณีศึกษา การเดินทางจากท่าเรือแหลมฉบัง ไป นิคมอุตสาหกรรมใน 3 จังหวัด อยุธยา ฉะเชิงเทรา สมุทรสาคร
3) หา บริเวณที่รถบรรทุกขับความเร็วเกินตามที่กฎหมายกับหนด ภายในภาคกลาง (80km/hr)

In [2]:
import pandas as pd


In [ ]:
gps_data = pd.read_csv(
    r"C:\Users\Documents\DCB Project\Raw Data\truckdata-00-06.csv",
    encoding="latin1"  # Use the correct encoding for your file
)
df = gps_data

In [ ]:
import pandas as pd
import os

# Set the folder path where your CSVs are located
folder_path = r'C:\Users\Documents\DCB Project\Raw Data'

# Find all CSV files in the folder
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Merge them into one DataFrame
df_list = []
for file in file_list:
    print(f"📥 Reading file: {file}")
    df = pd.read_csv(file, encoding='latin1', low_memory=False)  # Add header=0 if files have headers
    df_list.append(df)

# Combine all into one
merged_df = pd.concat(df_list, ignore_index=True)

# Save as Parquet
output_path = os.path.join(folder_path, 'merged_truckdata.parquet')
merged_df.to_parquet(output_path, index=False)

print("✅ All CSVs merged and saved to:", output_path)


📥 Reading file: C:\Users\SeniorProject\Documents\DCB Project\Raw Data\truckdata-00-06.csv
📥 Reading file: C:\Users\SeniorProject\Documents\DCB Project\Raw Data\truckdata-06-12.csv
📥 Reading file: C:\Users\SeniorProject\Documents\DCB Project\Raw Data\truckdata-12-18.csv
📥 Reading file: C:\Users\SeniorProject\Documents\DCB Project\Raw Data\truckdata-18-24.csv
✅ All CSVs merged and saved to: C:\Users\SeniorProject\Documents\DCB Project\Raw Data\merged_truckdata.parquet


Clean data

In [ ]:
df = pd.read_parquet(r"C:\Users\Documents\DCB Project\Raw Data\merged_truckdata.parquet")


In [ ]:
# 1. Initial row count
initial_rows = len(df)
print(f"Initial rows: {initial_rows:,}")

Initial rows: 338,540,013


In [11]:
# 2. Remove duplicates
before = len(df)
df = df.drop_duplicates()
print(f"Removed duplicates: {before - len(df):,} rows")

Removed duplicates: 94,073,784 rows


In [13]:
# 3. Drop missing values
before = len(df)
df = df.dropna()
print(f"Removed rows with missing values: {before - len(df):,}")

Removed rows with missing values: 0


In [14]:
df.head()

,unitid,utc_ts,lat,lon,speed,typecode
0,00300170000000T015820450148,2024-10-31 23:58:47,16.397420,102.813911,66,7
1,00300170000000T015820451022,2024-10-31 23:58:55,15.176015,104.843303,0,7
2,00300170000000T015820451233,2024-10-31 23:58:35,14.373698,100.021311,0,7
3,00300170000000T015820710006,2024-10-31 23:58:04,15.249035,100.204625,0,7
4,00300170000000T015820710404,2024-10-31 23:58:14,13.779333,100.686003,0,7


In [ ]:
# 4.Filter by lat/lon within Area Of Interest (AOI)
 
before = len(df)
lat_min, lat_max = 12.58, 14.54
lon_min, lon_max = 99.90, 102.05
df = df[(df["lat"].between(lat_min, lat_max)) & (df["lon"].between(lon_min, lon_max))]
print(f"Removed GPS outliers: {before - len(df):,} rows")

Removed GPS outliers: 141,824,816 rows


In [ ]:


# Define the path to the cleaned data folder
folder_path = r'C:\Users\Documents\DCB Project\CleanedData'

# Make sure the folder exists (optional, but good practice)
os.makedirs(folder_path, exist_ok=True)

# Define full output path for the cleaned parquet file
output_path = os.path.join(folder_path, 'cleaned_merged_truckdata.parquet')

# Save the cleaned DataFrame
df.to_parquet(output_path, index=False)